In [1]:
import pandas as pd

In [2]:
# Importing the dataset
df=pd.read_csv('train.csv')
df.sample(5)

,id,title,author,text,label
15124,15124,Summer Sweat: Hollywood Braces for Possible Wo...,John Binder,Hollywood may experience one of the worst slum...,0
9151,9151,Poor Families to Get More Help With Rent From ...,Nikita Stewart,In a settlement that could help thousands of f...,0
15259,15259,Watch: Senate Minority Leader Schumer Tears Up...,Pam Key,"Sunday at a press conference, Senate Minority ...",0
14691,14691,Nigerian Novelist Wonders Why Everyone Loves H...,Daniel Greenfield,Nigerian Novelist Wonders Why Everyone Loves H...,1
15987,15987,NaN,hotcocco,For that Statement alone I hope someone brings...,1


In [3]:
df.shape

(20800, 5)

In [4]:
# Dropping the NaN values
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
df.shape

(20800, 5)

In [6]:
# Seperating the dependent and the independent features
X=df.drop('label',axis=1)
Y=df['label']

In [7]:
# Tensorflow imporrs
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [8]:
# Onehot Encoding
messages=X.copy()
messages.reset_index(inplace=True)

In [9]:
import re
import nltk
import ssl
import tempfile
import os

ssl._create_default_https_context = ssl._create_unverified_context

download_dir = os.path.join(tempfile.gettempdir(), 'nltk')
nltk.data.path.append(download_dir)


In [10]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

from nltk.corpus import stopwords
nltk.download('stopwords')

for i in range(0, len(messages)):
    title=str(messages['title'][i])
    review=re.sub('[^a-zA-Z]', ' ', title)
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sohammandal01/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [12]:
voc_size=5000

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[411, 2055, 3964, 4908, 4261, 250, 3992, 4698, 374, 3327],
 [126, 3354, 2262, 517, 1025, 3726, 4851],
 [1225, 3583, 864, 2656],
 [89, 2948, 1203, 434, 2293, 3317],
 [1875, 1025, 1587, 2000, 3886, 3404, 1025, 1957, 1527, 694],
 [3862,
  3116,
  755,
  1922,
  1734,
  4904,
  2696,
  4481,
  1383,
  97,
  543,
  2907,
  4683,
  2125,
  4851],
 [2406, 2406, 2309, 2448, 4360, 2354, 4869, 1847, 2376, 2491, 2928, 3687],
 [3939, 3013, 1128, 632, 2918, 308, 42, 2903, 509, 3696, 2905],
 [162, 3778, 2697, 2481, 4904, 3604, 1854, 1173, 2040, 3393, 509, 3696, 2905],
 [3929, 2601, 4869, 3255, 650, 138, 4904, 4156, 509, 3696, 2905],
 [2274, 3396, 4294, 2089, 2434, 4812, 1357, 4983, 4904, 2123],
 [2352, 2299, 3697, 3376, 916, 759, 3711, 1513],
 [1606, 1121, 462, 3221, 1736, 3021, 1547, 2579, 4731, 185, 2663],
 [434, 744, 4261, 4812, 4904, 650],
 [2363, 92, 1846, 2618, 1163, 642, 4378, 4141, 1733],
 [3716, 3019, 1592, 4094, 4823, 232, 703, 509, 3696, 2905],
 [4862, 778, 2460, 2787, 1368, 509, 3696, 2

In [13]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4698  374 3327]
 [   0    0    0 ... 1025 3726 4851]
 [   0    0    0 ... 3583  864 2656]
 ...
 [   0    0    0 ...  509 3696 2905]
 [   0    0    0 ...  863 3096 3969]
 [   0    0    0 ...   83 3043 3697]]


In [14]:
# Model Creation
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
len(embedded_docs),Y.shape

(20800, (20800,))

In [16]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [17]:
X_final.shape,Y_final.shape

((20800, 20), (20800,))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [19]:
# Model Training
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8033 - loss: 0.4003 - val_accuracy: 0.9253 - val_loss: 0.1836
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9467 - loss: 0.1311 - val_accuracy: 0.9296 - val_loss: 0.1779
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9682 - loss: 0.0934 - val_accuracy: 0.9320 - val_loss: 0.2198
Epoch 4/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9760 - loss: 0.0678 - val_accuracy: 0.9256 - val_loss: 0.2335
Epoch 5/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9868 - loss: 0.0437 - val_accuracy: 0.9154 - val_loss: 0.2659
Epoch 6/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9913 - loss: 0.0299 - val_accuracy: 0.9244 - val_loss: 0.2935
Epoch 7/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9936 - loss: 0.0217 - val_accuracy: 0.9203 - val_loss: 0.3221
Epoch 8/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9958 - loss: 0.0148 - val_accu

In [20]:
# Performance Metrics and Accuracy
Y_pred_probs=model.predict(X_test)
Y_pred=np.round(Y_pred_probs).astype(int)

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
confusion_matrix(Y_test,Y_pred)

array([[3106,  343],
       [ 191, 3224]])

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9222027972027972